In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import copy
from PIL import Image
from io import BytesIO

import subprocess
import sys
from ast import literal_eval

sample_ply_data = o3d.data.DemoCropPointCloud()
tst_pointCloud = o3d.io.read_point_cloud(sample_ply_data.point_cloud_path)
#o3d.visualization.draw_geometries([tst_pointCloud])

def array_to_data(array):
    im = Image.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1,origin=[0, 0, 0])
    mesh_pc= mesh.sample_points_uniformly(number_of_points=1000000, use_triangle_normal=False)
    pointCloud+=mesh_pc
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.radians(RX), np.radians(RY), np.radians((RZ))))
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    #time.sleep(5)
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color

def Convert_message(string):
    return [int(s) for s in re.findall('[0-9]', string)]



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
clouds = [tst_pointCloud,tst_pointCloud]

In [ ]:
def Cluster_selection_gui(clouds):
    dat=[]  #to hold cluster's images
    pcd_combined = o3d.geometry.PointCloud()
    
    for current_cloud in clouds:
            color = fetch_cloud_image(current_cloud, RX=120, RY=0, RZ=220)  #different point clouds to be viewed
            dat.append(color)
    result = subprocess.run([sys.executable,  "Cluster_selection_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(dat))
    #selected_PC = Convert_message(result.stdout)
    OP = literal_eval(result.stdout)
    selected_PC = OP[0]
    selected_mode = OP[1]

    #for k in selected_PC:
    #    pcd_combined += clouds[k]  #combine selected pointclouds 
    #return pcd_combined
    return selected_PC, selected_mode



In [ ]:
selected_PC, selected_mode = Cluster_selection_gui(clouds)


# Front Page

In [2]:
def Front_gui(data_in=0):
    result = subprocess.run([sys.executable,  "Front_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(data_in))
    data_out = literal_eval(result.stdout)
    return data_out

data_out = Front_gui()
data_out


3

# Settings

In [ ]:
data_in = [1,0.01,0.13,0.0,0.05,0.0,True,200,0,0.05,10,0.01,3,0.8,True,True,0.3,3,0.1,True]

def Setting_gui(data_in):

    if len(data_in) < 20:
        print("Invalid data. Setting to default values.")
        data_in =[1,0.01,0.13,0.0,0.05,0.0,True,200,0,0.05,10,0.01,3,0.8,True,True,0.3,3,0.1,True] 
    result = subprocess.run([sys.executable,  "Settings_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(data_in))
    data_out = literal_eval(result.stdout)
    return data_out

data_out = Setting_gui(data_in)

status_flag = data_out[0]
samples = data_out[1] #1
spacing = data_out[2] #0.01
offset_y = data_out[3] #0.13
offset_z = data_out[4] #0.0
trim_base = data_out[5] #0.05
manual_offset = data_out[6] #0.0

Cluster_centered = data_out[7] #True
Cluster_idx = data_out[8] #200
Cluster_discard = data_out[9] #0 #disabled else put number of pts as threshold
eps = data_out[10] #0.05
min_points = data_out[11] #10
Cluster_trim = data_out[12] #0.01

TGT_coord_Samples = data_out[13] #3
TGT_final_trim = data_out[14] #0.8 #add max. euclidean distance away from robot's base (less than max reach of EEF) or z values less than...
TGT_reverse = data_out[15] #True
TGT_preview = data_out[16] #True
z_offset = data_out[17] #0.3
coord_skip = data_out[18] #3
TGT_motion_delay = data_out[19] #0.1
TGT_save = data_out[20] #True

print(status_flag, samples,spacing,offset_y,offset_z,trim_base,manual_offset,Cluster_centered,
            Cluster_idx,Cluster_discard,eps,min_points,Cluster_trim,TGT_coord_Samples,
            TGT_final_trim,TGT_reverse,TGT_preview,z_offset,coord_skip,TGT_motion_delay,TGT_save)

